In [1]:
import pandas as pd
import requests

URL_BASE_API_SDW = "https://sdw-2023-prd.up.railway.app/"

In [2]:
# Função para buscar dados dos clientes/usuarios
def get_user(id = "all"):

    argument = f"{URL_BASE_API_SDW}users"

    if id != "all":
        argument+= f"/{id}"

    response = requests.get(argument)

    return response.json() if response.status_code == 200 else None

In [3]:
# Busca dados de cada usuário
users = pd.DataFrame(get_user())

# Amplia as colunas para dados contidos nas colunas "account" e "card"
users = pd.concat([users.drop(["account"], axis=1), users["account"].apply(pd.Series).add_prefix("account_")], axis=1)
users = pd.concat([users.drop(["card"], axis=1), users["card"].apply(pd.Series).add_prefix('card_')], axis=1)

C:\Users\damas\AppData\Local\Temp\ipykernel_9580\3336957524.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  users = pd.concat([users.drop(["account"], axis=1), users["account"].apply(pd.Series).add_prefix("account_")], axis=1)
C:\Users\damas\AppData\Local\Temp\ipykernel_9580\3336957524.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  users = pd.concat([users.drop(["card"], axis=1), users["card"].apply(pd.Series).add_prefix('card_')], axis=1)


In [4]:
# Define base de dados apenas com variáveis de interesse
data = users[['id', 'name', 'account_balance', 'account_limit', 'card_limit']].fillna(0)

In [5]:
# Função que cria os grupos. Pode ser adaptado para uso de método de 'clustering'
def create_group(df, data_column, group_column):
    df = df.copy()

    # Calcula os percentis para determinar os limites dos grupos
    limite_classe_a = df[data_column].quantile(0.9)
    limite_classe_b = df[data_column].quantile(0.5)

    # Crie a nova coluna com base nos limites
    df[group_column] = 'negative'  # Inicialize como 'negativado'
    df.loc[df[data_column] > limite_classe_a, group_column] = 'A'
    df.loc[(df[data_column] <= limite_classe_a) & (df[data_column] > limite_classe_b), group_column] = 'B'
    df.loc[(df[data_column] <= limite_classe_b) & (df[data_column] >= 0), group_column] = 'C'

    return df

In [6]:
# Para cria coluna de classificação para cada coluna de interesse
for var in ['account_balance', 'account_limit', 'card_limit']:
    data = create_group(data, f"{var}",f"{var}_group")

Por fim, o dataframe gerado apresenta não só os valores atribuidos para balanço de conta, limite de conta e limite de cartão, mas também classificação destes valores de acordo com o grupo a que pertencem: A, caso estejam entre os 10% de maior valor; B, caso estejam entre os seguintes 40%; C caso estejam entre os 50% menores valores e não negativos, e negative, para aqueles que apresentaram valores negativos, que podem tanto representar pessoas com dívida no banco quanto pessoas com valores atribuidos de maneira errônea, caso o valor não pudesse ser negativo.